# Smart Reply: Automated Response Suggestion for Email
### Team Members:
### Kare Sindhu Gonda (sindhukaregonda@my.unt.edu)
### Pranathi Akula (PranathiAkula2@my.unt.edu)
### Vishnubotla Chandra Sekhar (Chandrasekharvishnubotla@my.unt.edu )

In [ ]:
pip install keras

In [ ]:
pip install tensorflow

In [2]:
import pandas as pd
import numpy as np
import re
from tensorflow.keras.preprocessing.sequence import pad_sequences
from keras.preprocessing.text import Tokenizer
from keras.models import Sequential
from keras.layers import Dense, LSTM, Embedding, Dropout
from keras.utils import to_categorical
from keras.layers import Embedding, LSTM, Dense, TimeDistributed
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from keras.layers import Embedding, LSTM, Dense, TimeDistributed

# Load the dataset
df = pd.read_csv("emails.csv")

# Display the first few rows of the DataFrame
print(df.head())

df = df.head(5000)

                       file                                            message
0     allen-p/_sent_mail/1.  Message-ID: <18782981.1075855378110.JavaMail.e...
1    allen-p/_sent_mail/10.  Message-ID: <15464986.1075855378456.JavaMail.e...
2   allen-p/_sent_mail/100.  Message-ID: <24216240.1075855687451.JavaMail.e...
3  allen-p/_sent_mail/1000.  Message-ID: <13505866.1075863688222.JavaMail.e...
4  allen-p/_sent_mail/1001.  Message-ID: <30922949.1075863688243.JavaMail.e...


In [3]:
#Extracts the Email Body 
import email

#parse email Body
def parse_func(x):
  b = email.message_from_string(x)
  body = ''
  if b.is_multipart():
    for payload in b.get_payload():
      body = body+ ' '
      body+=payload.get_payload()
  else:
    body+=b.get_payload()

  return body

#parse email Subject
def parse_func_subj(x):
  b = email.message_from_string(x)
  if b.get('Subject'):
    subj = b.get('Subject')
  else:
    subj = '-'
  return subj

df['Body'] = df['message'].apply(parse_func)
df['Subject'] = df['message'].apply(parse_func_subj)

df.tail()

,file,message,Body,Subject
3599,arnold-j/_sent_mail/605.,Message-ID: <3498983.1075857654351.JavaMail.ev...,how about benjys?,Re: ooops....
3600,arnold-j/_sent_mail/606.,Message-ID: <20051373.1075857654373.JavaMail.e...,just a vicious rumor... my birthday's not till...,Re:
3601,arnold-j/_sent_mail/607.,Message-ID: <16267167.1075857654394.JavaMail.e...,"it was almost worth buying a ticket\n\n\n\n\n""...",Re: ooops....
3602,arnold-j/_sent_mail/608.,Message-ID: <773676.1075857654425.JavaMail.eva...,---------------------- Forwarded by John Arnol...,Enron Mentions
3603,arnold-j/_sent_mail/609.,Message-ID: <17621797.1075857654714.JavaMail.e...,---------------------- Forwarded by John Arnol...,Fw: ETKT Confirmation -


In [4]:
# Clean the text and remove special characters
def clean_text(text):
    text = re.sub(r"[^a-zA-Z0-9]", " ", text)
    text = text.lower()
    return text

df["Body"] = df["Body"].apply(clean_text)

In [5]:
text = df['Body'].values

In [6]:
# Tokenize the body column
tokenizer = Tokenizer(num_words=100)
tokenizer.fit_on_texts(df['Body'].values)

# Convert text to sequences
sequences = tokenizer.texts_to_sequences(df['Body'].values)

# Pad sequences
max_seq_len = max([len(seq) for seq in sequences])
X = pad_sequences(sequences, maxlen=max_seq_len, padding='pre', truncating='pre')

# Prepare input and target
input_seq = X[:, :-1]
target_seq = X[:, 1:]
y = pad_sequences(to_categorical(X[:, 1:], num_classes=tokenizer.num_words), maxlen=max_seq_len, padding='pre')

# Split data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)


In [25]:
# Define the model architecture
model = Sequential()
model.add(Embedding(input_dim=tokenizer.num_words, output_dim=32, input_shape=(None,)))
model.add(LSTM(units=64, return_sequences=True))
model.add(TimeDistributed(Dense(tokenizer.num_words, activation='softmax')))

# Compile the model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Train the model
model.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=5, batch_size=16)

# Save the tokenizer and the model
tokenizer_json = tokenizer.to_json()
with open('tokenizer.json', 'w', encoding='utf-8') as f:
    f.write(tokenizer_json)
model.save('lstm_model.h5')

Epoch 1/5
181/181 [==============================] - 179s 956ms/step - loss: 0.8312 - accuracy: 0.9350 - val_loss: 0.2891 - val_accuracy: 0.9516
Epoch 2/5
181/181 [==============================] - 154s 851ms/step - loss: 0.2411 - accuracy: 0.9524 - val_loss: 0.2258 - val_accuracy: 0.9540
Epoch 3/5
181/181 [==============================] - 154s 854ms/step - loss: 0.2197 - accuracy: 0.9537 - val_loss: 0.2115 - val_accuracy: 0.9541
Epoch 4/5
181/181 [==============================] - 155s 857ms/step - loss: 0.2094 - accuracy: 0.9546 - val_loss: 0.2053 - val_accuracy: 0.9551
Epoch 5/5
181/181 [==============================] - 154s 851ms/step - loss: 0.2040 - accuracy: 0.9553 - val_loss: 0.1987 - val_accuracy: 0.9572


In [9]:
#testing the Models
from tensorflow.keras.preprocessing.sequence import pad_sequences
from keras.models import load_model
from keras_preprocessing.text import tokenizer_from_json
from tensorflow.keras.preprocessing.sequence import pad_sequences
import numpy as np

# Load the tokenizer
with open('tokenizer.json', 'r') as f:
    tokenizer = tokenizer_from_json(f.read())

# Load the model
model = load_model('lstm_model.h5')

# Load test set
test_sequences = tokenizer.texts_to_sequences(df['Body'].values)
# Pad sequences
max_seq_len = 100
test_padded = pad_sequences(test_sequences, maxlen=max_seq_len, padding='post', truncating='post')

# Generate predictions
predictions = model.predict(test_padded)

# Print results
for i in range(len(df)):
    #print('Input: {}'.format(df['Body'].iloc[i]))
    predicted_index = np.argmax(predictions[i])
    predicted_word = tokenizer.index_word[predicted_index]
    #print('Predicted Next Word: {}'.format(predicted_word))
    #print('\n')



113/113 [==============================] - 4s 25ms/step


In [40]:
import numpy as np
import string

# Load the tokenizer
with open('tokenizer.json', 'r') as f:
    tokenizer = tokenizer_from_json(f.read())

# Load the model
model = load_model('lstm_model.h5')


# Input text
input_text = 'Do you need any help?'

# Preprocess the input text
input_text = input_text.lower()
input_text = input_text.translate(str.maketrans('', '', string.punctuation))
tokens = tokenizer.texts_to_sequences([input_text])[0]
tokens = pad_sequences([tokens], maxlen=10)

# Predict the next word
probabilities = model.predict(tokens)[0]
predicted_index = np.argmax(probabilities)
predicted_word = tokenizer.index_word[predicted_index]

# Print the predicted word
print(predicted_word)


1/1 [==============================] - 0s 429ms/step
program


In [41]:
# Input text
input_text = 'Let me know about your status on'

# Preprocess the input text
input_text = input_text.lower()
input_text = input_text.translate(str.maketrans('', '', string.punctuation))
tokens = tokenizer.texts_to_sequences([input_text])[0]
tokens = pad_sequences([tokens], maxlen=10)

# Predict the next word
probabilities = model.predict(tokens)[0]
predicted_index = np.argmax(probabilities)
predicted_word = tokenizer.index_word[predicted_index]

# Print the predicted word
print(predicted_word)

1/1 [==============================] - 0s 25ms/step
program


In [42]:
# Input text
input_text = 'Please support team on'

# Preprocess the input text
input_text = input_text.lower()
input_text = input_text.translate(str.maketrans('', '', string.punctuation))
tokens = tokenizer.texts_to_sequences([input_text])[0]
tokens = pad_sequences([tokens], maxlen=10)

# Predict the next word
probabilities = model.predict(tokens)[0]
predicted_index = np.argmax(probabilities)
predicted_word = tokenizer.index_word[predicted_index]

# Print the predicted word
print(predicted_word)

1/1 [==============================] - 0s 26ms/step
floor
